In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [2]:
#github pathing 
"""
dtravel = pd.read_csv("../data/travelq.csv")
otravel = pd.read_csv("../data/orgdetail.csv")
"""
# Load datasets
dtravel = pd.read_csv("travelq.csv")
otravel = pd.read_csv("orgdetail.csv")

In [3]:
# Basic Cleaning
# Standardizing columns
numeric_col = ["airfare", "other_transport", "lodging", "meals", "other_expenses", "total"]
dtravel[numeric_col] = dtravel[numeric_col].apply(pd.to_numeric, errors='coerce').round(2)

non_numeric_col = dtravel.select_dtypes(exclude=['number']).columns
dtravel[non_numeric_col] = dtravel[non_numeric_col].astype(str)

In [4]:
# Remove unnecessary columns
def remove_columns(dtravel, columns_to_remove):
    dtravel = dtravel.drop(columns=columns_to_remove, errors='ignore')
    print(f"Removed columns: {', '.join(columns_to_remove)}")
    return dtravel

columns_to_remove = [
    'disclosure_group', 'title_fr', 'purpose_fr', 'destination_fr', 
    'additional_comments_en', 'additional_comments_fr'
]
dtravel = remove_columns(dtravel, columns_to_remove)

Removed columns: disclosure_group, title_fr, purpose_fr, destination_fr, additional_comments_en, additional_comments_fr


In [5]:
# Remove outliers using IQR method
def remove_outliers(dtravel, numerical_columns):
    for column in numerical_columns:
        Q1 = dtravel[column].quantile(0.25)
        Q3 = dtravel[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        dtravel.loc[(dtravel[column] < lower_bound) | (dtravel[column] > upper_bound), column] = np.nan
        print(f"Processed Outliers for {column} (Outliers replaced with NaN)")
    
    return dtravel 

numerical_columns = dtravel.select_dtypes(include=['number']).columns.tolist()
dtravel = remove_outliers(dtravel, numerical_columns)


Processed Outliers for airfare (Outliers replaced with NaN)
Processed Outliers for other_transport (Outliers replaced with NaN)
Processed Outliers for lodging (Outliers replaced with NaN)
Processed Outliers for meals (Outliers replaced with NaN)
Processed Outliers for other_expenses (Outliers replaced with NaN)
Processed Outliers for total (Outliers replaced with NaN)


In [6]:
# Clean the owner_org_title column
def clean_owner_org_title(dtravel):
    dtravel['owner_org_title'] = dtravel['owner_org_title'].str.split(' \\| ').str[0]
    return dtravel

dtravel = clean_owner_org_title(dtravel)
print("Removed the french out of 'owner_org_title'")

Removed the french out of 'owner_org_title'


In [7]:
# Merge datasets using a LEFT JOIN on 'owner_org_title' and 'owner_org'
mmerge = dtravel.merge(otravel, on=["owner_org_title", "owner_org"], how="left")

In [8]:
# Save the merged dataset with today's date in the filename
date_str = date.today().strftime("%Y-%m-%d")
merged_file_name = f"merged_travel_data_{date_str}.csv"
mmerge.to_csv(merged_file_name, index=False)
print(f"Merged dataset saved as: {merged_file_name}")

Merged dataset saved as: merged_travel_data_2025-02-28.csv


# Explanation of Variables
- `dtravel`: The main dataset containing travel details.
- `otravel`: The secondary dataset containing organization details.
- `numeric_col`: List of columns that should be treated as numeric.
- `non_numeric_col`: List of columns that should be treated as strings.
- `columns_to_remove`: List of columns to be removed from the dataset.
- `numerical_columns`: List of numeric columns for outlier detection.
- `mmerge`: The merged dataset combining `dtravel` and `otravel`.

# Explanation of Compound Key
A compound key is a combination of multiple columns that together create a unique identifier for rows in a DataFrame. 
In this case, the compound key is created using 'owner_org_title' and 'owner_org' columns. 
This ensures that each row in the merged dataset is uniquely identified by the combination of these two columns.


# Explanation of Join vs Merge
- **Join**: A join operation combines rows from two or more tables based on a related column between them. 
  It is typically used in SQL and can be thought of as a way to combine rows from different tables based on a condition.
  
- **Merge**: A merge operation in pandas is similar to a join but is more flexible. It allows for more complex combinations 
  of dataframes, including different types of joins (inner, outer, left, right). In this case, we used a LEFT JOIN to 
  ensure that all rows from the `dtravel` dataset are preserved, even if there are no matching rows in the `otravel` dataset.
